# Seasonal Trading Plan Project
### This project seeks to figure out which stocks consistently go up over 30, 60 or 90 day periods, year after year, at least 80% of the time. It turns out there are many of them.

In this notebook, I created a process to download full historical EOD price data on each of the S&P 500 index components and analyze the historical patterns to find situations where seasonal trends can be taken advantage of over the course of a year.

For each stock, I calculated the following statistics:

Holding Period:  The number of days held in each rolling period - 30, 60 and 90 days each.

% Up Rows:  The percentage of rolling periods where, each year, the stock went up at least 80% of the time.

Max Up Return: The highest return for each rolling period where the return was positive.

Min Up Return: The lowest return for each rolling period where the return was positive.

Avg Up Return:  The average return for each rolling period where the return was positive.

Avg Down Return:  The average return for each rolling period where the return was negative.

Expected Return: The average return for all rolling periods.

StDev Returns: The average standard deviation of returns for all rolling periods. This means that the expected return should be within this +/- range from the average 67% of the time.

% Downside:  The expected average return minus the standard deviation. This means that you should at least exceed this return 67% of the time. This could provide guidance for setting stop-loss levels.

Worst Return:  The worst return of all rolling periods.

Least Pain Pt:  The time interval that offers the best case 67% downside within the sweet spot period. This is used to pinpoint the lowest risk time frame.

Best Begin Date:  The recommended entry date for best risk/reward scenario.

Best End Date:  The recommended exit date for best risk/reward scenario.

Max Consec 80%+:  The longest consecutive number of rolling periods above 80% up years. You want to see more that a few consecutive up years over the threshold to provide assurance that the trend is robust.

Total Years:  The number of years of historical data available for each stock. I filtered for stocks with 10+ years of history.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf
from pandas_datareader import data as pdr
import xlsxwriter

In [2]:
# This is the path to where I store this notebook and downloaded files. You need to change this to a convenient
# spot on your own hard drive.

my_path = '/Users/bnsheehy/Documents/Investments/Seasonal_Analyses/Python'
threshold = 0.80

In [3]:
# You need to go to Yahoo and download a list of the S&P 500 components. Make sure to save it to
# a CSV file with column headers that include "Symbol", "Date" and "Close" 
file_sp500_tickers = my_path + "/SP500_TickerList.csv"

In [4]:
# Upload a list of the S&P 500 components downloaded from Yahoo.

df_sp500_tickers = pd.read_csv (file_sp500_tickers)
df_sp500_tickers.head()

,Symbol,Company Name,GICS Sector,GICS Sub Industry
0,A,Agilent Technologies Inc,Health Care,Health Care Equipment
1,AAL,American Airlines Group,Industrials,Airlines
2,AAP,Advance Auto Parts,Consumer Discretionary,Automotive Retail
3,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals"
4,ABBV,Rickers,Health Care,Pharmaceuticals


In [5]:
# This module loops through the S&P 500 tickers, downloads the data from Yahoo and creates a separate CSV 
# file of historical data for each ticker (e.g. AAPL.csv).
# Skip this routine if you already have the CSV files available.

for index, ticker in df_sp500_tickers.iterrows():
    global df
    
    my_ticker = ticker['Symbol']
    
    yf_ticker = yf.Ticker(my_ticker)
    data = yf_ticker.history(period="max")
    df = pd.DataFrame(data)
    df.reset_index(level=0, inplace=True)
    df['Symbol'] = my_ticker
    df = df[['Symbol','Date','Close']]
    df.drop_duplicates(subset ="Date", keep = 'first', inplace = True) #Yahoo has a tendency to duplicate the last row.
    df.to_csv(path_or_buf = my_path + "/data/" + my_ticker +".csv", index=False)
    

In [6]:
# Creates the dataframe container for the stats data.

df_tradelist = pd.DataFrame(index=[], columns=['my_ticker', 'hold_per', 'pct_uprows', 'max_up_return', 'min_up_return', 'avg_up_return', 'avg_down_return', 'exp_return', 'stdev_returns', 'pct_downside', 'worst_return', 'least_pain_pt', 'total_years', 'max_consec_beat', 'best_buy_date', 'best_sell_date', 'analyzed_years'])

df_tradelist.head()

,my_ticker,hold_per,pct_uprows,max_up_return,min_up_return,avg_up_return,avg_down_return,exp_return,stdev_returns,pct_downside,worst_return,least_pain_pt,total_years,max_consec_beat,best_buy_date,best_sell_date,analyzed_years


In [7]:
# Convert prices to holding period returns based on 20 trading days per month.

def convert_prices_to_periods():
    
    global dperiods
    global dfr
        
    dfr = df.pct_change(periods = dperiods)
    dfr.reset_index(level=0, inplace=True)
    dfr.rename(columns={'Close':'Returns'}, inplace=True)
    dfr = dfr.round(4)

In [8]:
# Separate out the date column into separate month, year and day values.

def separate_date_column():
    
    global dfr
    
    dfr['Month'] = pd.DatetimeIndex(dfr['Date']).month
    dfr['Day'] = pd.DatetimeIndex(dfr['Date']).day
    dfr['Year'] = pd.DatetimeIndex(dfr['Date']).year
    dfr['M-D'] = dfr['Month'].astype(str)+'-'+dfr['Day'].astype(str)
    pd.set_option('display.max_rows', len(dfr))

In [9]:
# Pivot the table to show years across the top and Month-Day values in the first column on the left.

def pivot_the_table():
    
    global dfr_pivot
    
    dfr_pivot = dfr.pivot(index='M-D', columns='Year', values='Returns')
    dfr_pivot.reset_index(level=0, inplace=True)
    dfr_pivot = pd.DataFrame(dfr_pivot)
    dfr_pivot.columns.name="Index"

    # The pivot operation created empty cells for weekends and holiday, so I filled them with EOD values from
    # the previous trading day.
    dfr_pivot.fillna(method='ffill', inplace=True)
    
    # As of this date, 1/22/2020, we are only evaluating results through 12/31/2019, so we will drop the
    # 2020 year column.
    if 2020 in dfr_pivot.columns:
        dfr_pivot.drop(2020, axis=1, inplace=True)
    

In [10]:
# Add additional calculated columns to facilitate statistic calculations for each stock.

def add_calculated_items():
    
    global dfr_pivot
    global lookback
    global start
    
    # The lookback figure is the number (must be an integer) of years back from last year (2019) that you want to include in
    # analysis, i.e. the calculations below. It's probably a good idea to keep it at 20 years or less
    # to reflect more recent market conditions.
    lookback = 20
    start = 1
    
    if lookback > len(dfr_pivot.columns) - 1:
        start = 1
    else:
        start = len(dfr_pivot.columns) - lookback
    
    dfr_pivot['YearCount'] = dfr_pivot.count(axis=1, numeric_only=True)
    dfr_pivot['Lookback'] = lookback
    dfr_pivot['UpCount'] = dfr_pivot[dfr_pivot.iloc[:,start:len(dfr_pivot.columns)-2] > 0].count(axis=1)
    dfr_pivot['DownCount'] = dfr_pivot[dfr_pivot.iloc[:,start:len(dfr_pivot.columns)] < 0].count(axis=1)
    dfr_pivot['PctUp'] = dfr_pivot['UpCount']/dfr_pivot['Lookback']
    dfr_pivot['PctDown'] = dfr_pivot['DownCount']/dfr_pivot['Lookback']
    dfr_pivot['AvgReturn'] = dfr_pivot.iloc[:,start:len(dfr_pivot.columns)-6].mean(axis=1)
    dfr_pivot['StDevReturns'] = dfr_pivot.iloc[:,start:len(dfr_pivot.columns)-7].std(axis=1)
    dfr_pivot['67PctDownside'] = dfr_pivot['AvgReturn']-dfr_pivot['StDevReturns']
    dfr_pivot['MaxReturn'] = dfr_pivot.iloc[:,start:len(dfr_pivot.columns)-9].max(axis=1)
    dfr_pivot['MinReturn'] = dfr_pivot.iloc[:,start:len(dfr_pivot.columns)-10].min(axis=1)

In [11]:
# Add a fictional date column in Python date/time format so the table can be sorted by date. Then sort by Date.
# Reset the index and round the float values to 4 decimals.

def sortbydate_resetindex_export():
    
    global dfr_pivot
    
    dfr_pivot['Date'] = '2000-' + dfr_pivot['M-D'].astype(str)
    dfr_pivot['Date'] = pd.to_datetime(dfr_pivot['Date'], infer_datetime_format=True)
    dfr_pivot.sort_values(by='Date',ascending=True, inplace=True)
        
    dfr_pivot.reset_index(inplace=True)
    dfr_pivot = dfr_pivot.round(4)


In [12]:
# Calculate the trading statistics for the rolling holding periods for the stock.

def calc_trading_stats():
    
    global interval
    global dfr_pivot
    global pct_uprows
    global max_up_return
    global min_up_return
    global avg_up_return
    global avg_down_return
    global exp_return
    global stdev_returns
    global pct_downside
    global worst_return
    global least_pain_pt
    global total_years
    global n_consec
    global max_n_consec
    global max_consec_beat
    global best_sell_date
    global best_buy_date
    global analyzed_years
    global lookback
    
    pct_uprows = (dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, 'PctUp'].count() / dfr_pivot.loc[:, 'PctUp'].count()).astype(float).round(4)
    max_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, 'MaxReturn'].max()
    min_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, 'MinReturn'].min()
    avg_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > 0.5, 'AvgReturn'].mean()
    avg_up_return = np.float64(avg_up_return).round(4)
    avg_down_return = dfr_pivot.loc[dfr_pivot['PctDown'] > 0.5, 'AvgReturn'].mean()
    avg_down_return = np.float64(avg_down_return).round(4)
    exp_return = dfr_pivot['AvgReturn'].mean().round(4)
    stdev_returns = dfr_pivot['StDevReturns'].mean()
    stdev_returns = np.float64(stdev_returns).round(4)
    worst_return = dfr_pivot['MinReturn'].min()
    pct_downside = exp_return - stdev_returns
    pct_downside = np.float64(pct_downside).round(4)
    least_pain_pt = dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, '67PctDownside'].max()
    total_years = dfr_pivot['YearCount'].max()
    analyzed_years = lookback
    
    n_consec = 0
    max_n_consec = 0

    for x in dfr_pivot['PctUp']:
        if (x > threshold):
            n_consec += 1
        else: # check for new max, then start again from 1
            max_n_consec = max(n_consec, max_n_consec)
            n_consec = 1

    max_consec_beat = max_n_consec

    try:
        best_sell_date = dfr_pivot.loc[dfr_pivot['67PctDownside'] == least_pain_pt, 'M-D'].iloc[0]
    except:
        best_sell_date = "nan"

    try:
        row = dfr_pivot.loc[dfr_pivot['M-D'] == best_sell_date, 'M-D'].index[0] - interval
        col = dfr_pivot.columns.get_loc('M-D')
        best_buy_date = dfr_pivot.iloc[row,col]
    except:
        best_buy_date = "nan"

In [13]:
# If the pct_uprows and history conditions are met, then create the array of stat values and append 
# it to the recommended trade list.

def filter_and_append_stats():
    
    global statsdata
    global df_statsdata
    global df_tradelist
    
    # Save the stats data separately to export to Excel for further research on each ticker if desired.
    statsdata = np.array([my_ticker, hold_per, pct_uprows, max_up_return, min_up_return, avg_up_return, avg_down_return, exp_return, stdev_returns, pct_downside, worst_return, least_pain_pt, total_years, max_consec_beat, best_buy_date, best_sell_date, analyzed_years])
    df_statsdata = pd.DataFrame(statsdata.reshape(-1, len(statsdata)), columns=['my_ticker', 'hold_per', 'pct_uprows', 'max_up_return', 'min_up_return', 'avg_up_return', 'avg_down_return', 'exp_return', 'stdev_returns', 'pct_downside', 'worst_return', 'least_pain_pt', 'total_years', 'max_consec_beat', 'best_buy_date', 'best_sell_date', 'analyzed_years'])
    
    if pct_uprows > 0.1:
        if total_years > 9:
            df_tradelist = df_tradelist.append(dict(zip(df_tradelist.columns, statsdata)), ignore_index=True)
            

In [14]:
# This module grabs each ticker file, transforms it and calculates the statistics needed for a 90 day holding period.

def calc_3month_returns():
    
    global dfr
    global dfr_pivot
    global df_tradelist
    global dfr_3mo
    global df_statsdata_3mo
    global threshold
    global hold_per
    global dperiods
    global interval
    
    dperiods = 60
    hold_per = "3 Mos"
    interval = 90
    
    convert_prices_to_periods()
    
    separate_date_column()

    pivot_the_table()

    add_calculated_items()

    sortbydate_resetindex_export()
    
    # Export the pivot table to CSV for further research if desired.
    #dfr_pivot.to_csv(path_or_buf = my_path + "/data/" + my_ticker + "_dfr_pivot_3mo.csv", index=False)
    
    # Save dfr_pivot to separate dataframe for exporting to Excel
    dfr_3mo = pd.DataFrame(dfr_pivot)

    calc_trading_stats()
    
    filter_and_append_stats()
    
    # Save statsdata to separate dataframe for exporting to Excel
    df_statsdata_3mo = df_statsdata.copy()
    

In [15]:
# This module grabs each ticker file, transforms it and calculates the statistics needed for a 60 day holding period.

def calc_2month_returns():
    
    global dfr
    global dfr_pivot
    global df_tradelist
    global dfr_2mo
    global df_statsdata_2mo
    global threshold
    global hold_per
    global dperiods
    global interval
    
    dperiods = 40
    hold_per = "2 Mos"
    interval = 60

    convert_prices_to_periods()
    
    separate_date_column()

    pivot_the_table()

    add_calculated_items()

    sortbydate_resetindex_export()
    
    # Export the pivot table to CSV for further research if desired.
    #dfr_pivot.to_csv(path_or_buf = my_path + "/data/" + my_ticker + "_dfr_pivot_2mo.csv", index=False)
    
    # Save dfr_pivot to separate dataframe for exporting to Excel
    dfr_2mo = pd.DataFrame(dfr_pivot)

    calc_trading_stats()
        
    filter_and_append_stats()
     
    # Save statsdata to separate dataframe for exporting to Excel
    df_statsdata_2mo = df_statsdata.copy()
            

In [16]:
# This module grabs each ticker file, transforms it and calculates the statistics needed for a 30 day holding period.

def calc_1month_returns():
    
    global dfr
    global dfr_pivot
    global df_tradelist
    global dfr_1mo
    global df_statsdata_1mo
    global threshold
    global hold_per
    global dperiods
    global interval
    
    dperiods = 20
    hold_per = "1 Mo"
    interval = 30

    convert_prices_to_periods()
    
    separate_date_column()

    pivot_the_table()

    add_calculated_items()

    sortbydate_resetindex_export()
    
    # Export the pivot table to CSV for further research if desired.
    #dfr_pivot.to_csv(path_or_buf = my_path + "/data/" + my_ticker + "_dfr_pivot_1mo.csv", index=False)
    
    # Save dfr_pivot to separate dataframe for exporting to Excel
    dfr_1mo = pd.DataFrame(dfr_pivot)

    calc_trading_stats()
        
    filter_and_append_stats()
    
    # Save statsdata to separate dataframe for exporting to Excel
    df_statsdata_1mo = df_statsdata.copy()
            

In [17]:
# Build and export an Excel file for each ticker using XlsxWriter

def export_to_excel():
    
    excel_file_path = my_path + "/data/" + my_ticker + ".xlsx"
    
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(excel_file_path, engine='xlsxwriter')

    # Convert the dataframe to an XlsxWriter Excel object.
    df_statsdata_1mo.to_excel(writer, sheet_name='Stats', index=False)
    df_statsdata_2mo.to_excel(writer, sheet_name='Stats', startrow=2, header=False, index=False)
    df_statsdata_3mo.to_excel(writer, sheet_name='Stats', startrow=3, header=False, index=False)
    dfr_1mo.to_excel(writer, sheet_name='1 Mo Returns', index=False)
    dfr_2mo.to_excel(writer, sheet_name='2 Mo Returns', index=False)
    dfr_3mo.to_excel(writer, sheet_name='3 Mo Returns', index=False)

    # Get the xlsxwriter objects from the dataframe writer object.
    workbook  = writer.book
    worksheet1 = writer.sheets['Stats']
    worksheet2 = writer.sheets['1 Mo Returns']
    worksheet3 = writer.sheets['2 Mo Returns']
    worksheet4 = writer.sheets['3 Mo Returns']
    
    # Add conditional formatting to highlight positive returns in green
    end_column = dfr_1mo.columns.get_loc("YearCount")
    grn_format = workbook.add_format({'bg_color':   '#C6EFCE','font_color': '#006100'})
    worksheet2.conditional_format(1, 2, 365, end_column - 1,{'type':'cell','criteria':'>','value':0,'format':grn_format})
    worksheet3.conditional_format(1, 2, 365, end_column - 1,{'type':'cell','criteria':'>','value':0,'format':grn_format})
    worksheet4.conditional_format(1, 2, 365, end_column - 1,{'type':'cell','criteria':'>','value':0,'format':grn_format})
    
    # Freeze panes for scrolling
    worksheet2.freeze_panes(1, 2)
    worksheet3.freeze_panes(1, 2)
    worksheet4.freeze_panes(1, 2)
    
    # Save the file
    writer.save()


In [18]:

# Read CSV files by ticker, transform and extract stats from each one.

for index, ticker in df_sp500_tickers.iterrows():
    
    global df
    global dfr
    
    my_ticker = ticker['Symbol']

    df = pd.read_csv (my_path + "/data/" + my_ticker + ".csv")
    df.set_index('Date', inplace=True)
    df = df['Close']
    df = pd.DataFrame(df, columns=['Close'])
    
    calc_1month_returns()
    
    calc_2month_returns()
    
    calc_3month_returns()
    
    export_to_excel()


In [19]:
# Make a copy and convert the trade list to a Pandas dataframe.
df_tradelist_copy = df_tradelist.copy()
df_tradelist = pd.DataFrame(df_tradelist)

In [20]:
#df_tradelist.to_csv(path_or_buf = my_path + "/df_tradelist.csv", index=False)
#df_tradelist_copy.to_csv(path_or_buf = my_path + "/df_tradelist_copy.csv", index=False)

In [21]:
# Clean it up by removing rows with NaN's and infinity values and dropping duplicates.
df_tradelist.replace("inf", np.nan, inplace=True)
df_tradelist.dropna(inplace=True)
df_tradelist = df_tradelist[~df_tradelist.max_up_return.str.contains("nan")]
df_tradelist = df_tradelist[~df_tradelist.avg_down_return.str.contains("nan")]
df_tradelist.sort_values(by=['pct_uprows'], ascending=False)
df_tradelist.drop_duplicates(subset ="my_ticker", keep = 'first', inplace = True) 

In [22]:
# Preview the end of the trade list to see how many rows and make sure the data is showing up where it is supposed to.

df_tradelist.tail(10)

,my_ticker,hold_per,pct_uprows,max_up_return,min_up_return,avg_up_return,avg_down_return,exp_return,stdev_returns,pct_downside,worst_return,least_pain_pt,total_years,max_consec_beat,best_buy_date,best_sell_date,analyzed_years
39,O,3 Mos,0.1433,0.3028,-0.2535,0.0472,0.0193,0.0468,0.0926,-0.0458,-0.3844,0.0091,25,18,1-10,4-9,20
40,PGR,3 Mos,0.1295,0.8968,-0.1178,0.0589,-0.0133,0.0424,0.118,-0.0756,-0.4189,0.0174,40,28,2-1,5-1,20
41,ROK,2 Mos,0.1047,0.4501,-0.318,0.0437,-0.0098,0.0315,0.123,-0.0915,-0.5223,0.0277,38,10,10-27,12-27,20
42,RSG,2 Mos,0.1543,0.5248,-0.3205,0.0339,-0.0196,0.0268,0.0819,-0.0551,-0.4356,-0.0003,22,8,3-1,4-30,20
44,SNA,2 Mos,0.1295,0.6487,-0.4425,0.0363,-0.0196,0.024,0.0992,-0.0752,-0.4913,0.0065,47,11,10-19,12-18,20
45,UDR,3 Mos,0.1791,0.6013,-0.4676,0.0395,0.0035,0.0381,0.1115,-0.0734,-0.55,0.0113,40,34,2-20,5-20,20
46,UNP,2 Mos,0.1019,0.3381,-0.3516,0.0367,0.0037,0.0333,0.0875,-0.0542,-0.3516,-0.0083,40,9,4-11,6-10,20
48,VFC,3 Mos,0.1818,0.3731,-0.2119,0.0601,0.0168,0.0595,0.1137,-0.0542,-0.4707,0.0713,38,38,2-1,5-1,20
49,VLO,3 Mos,0.1074,0.7542,-0.3603,0.0966,-0.0127,0.0622,0.1766,-0.1144,-0.5954,0.0195,38,24,11-26,2-26,20
51,YUM,2 Mos,0.1267,0.5137,-0.1324,0.0389,-0.0036,0.0288,0.0877,-0.0589,-0.3573,0.0266,23,11,3-2,5-1,20


In [23]:
df_tradelist.head()
#df_tradelist.shape

,my_ticker,hold_per,pct_uprows,max_up_return,min_up_return,avg_up_return,avg_down_return,exp_return,stdev_returns,pct_downside,worst_return,least_pain_pt,total_years,max_consec_beat,best_buy_date,best_sell_date,analyzed_years
0,ALL,2 Mos,0.1102,0.7888,-0.1366,0.0243,-0.0049,0.0202,0.0988,-0.0786,-0.601,0.0071,27,13,2-9,4-9,20
1,AME,2 Mos,0.1102,0.3975,-0.3156,0.039,-0.0032,0.0364,0.0851,-0.0487,-0.4141,0.016,36,6,10-24,12-23,20
3,BDX,3 Mos,0.1653,0.3388,-0.1588,0.0377,0.0035,0.0353,0.0839,-0.0486,-0.3203,0.031,47,25,11-24,2-24,20
4,BKNG,2 Mos,0.1074,1.6303,-0.2764,0.064,0.0034,0.0502,0.2522,-0.202,-0.8837,0.0453,21,10,2-5,4-5,20
6,BLK,3 Mos,0.1129,0.4583,-0.5222,0.0588,0.0311,0.0565,0.1422,-0.0857,-0.53,0.0069,21,7,10-23,1-23,20


In [24]:
# Export the trade list to CSV files for execution and/or further research if desired.
df_tradelist.to_csv(path_or_buf = my_path + "/df_tradelist.csv", index=False)